# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

В следующей ячейке сохраним все инструменты, которые будем использовать:

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter("ignore")

Далее откроем файлы и посмотрим данные:

In [16]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')
print(df1.info())
print(df2.info())
print(df3.info())
print(df1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

Данных много посмотрим число уникальных значений в столбце product:

In [17]:
print(len(df1['product'].unique()))
print(len(df2['product'].unique()))
print(len(df3['product'].unique()))

100000
12
100000


В первом и втором файле уникальных значений слишком много, так же по данным из проверки функцией info, данные в этом столбце имеют формат float, возможно стоит перевести их в формат int во всех файлах и это уменьшит кол-во значений, так же переведет в удобный формат для будущей модели. 

In [18]:
df1['product'] = df1['product'].astype(int)
df2['product'] = df2['product'].astype(int)
df3['product'] = df3['product'].astype(int)
print(len(df1['product'].unique()))
print(len(df2['product'].unique()))
print(len(df3['product'].unique()))

186
12
191


Далее разделим датасеты на выборки и сразу масштабируем признаки:

In [19]:
target_1 = df1['product']
features_1 = df1.drop(['product', 'id'], axis=1)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)
numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])

In [20]:
target_2 = df2['product']
features_2 = df2.drop(['product', 'id'], axis=1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

scaler = StandardScaler()
scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])

In [21]:
target_3 = df3['product']
features_3 = df3.drop(['product', 'id'], axis=1)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=12345)

scaler = StandardScaler()
scaler.fit(features_train_3[numeric])
features_train_3[numeric] = scaler.transform(features_train_3[numeric])
features_valid_3[numeric] = scaler.transform(features_valid_3[numeric])

**Вывод:** Данные готовы к обучению модели. 

## Обучение и проверка модели

Создадим модели на данных каждого региона посчитаем средний предсказанный объем сырья и rsme:

In [22]:
model_lr_1 = LinearRegression()
model_lr_1.fit(features_train_1, target_train_1) 
predictions_valid_lr_1 = model_lr_1.predict(features_valid_1) 
predictions_mean = predictions_valid_lr_1.mean()
result_lr = mean_squared_error(target_valid_1, predictions_valid_lr_1)**0.5
print('Средний запас сырья:', predictions_mean, 'rmse =', result_lr)

Средний запас сырья: 92.0933978221753 rmse = 37.58106136304993


In [23]:
model_lr_2 = LinearRegression()
model_lr_2.fit(features_train_2, target_train_2) 
predictions_valid_lr_2 = model_lr_2.predict(features_valid_2) 
predictions_mean = predictions_valid_lr_2.mean()
result_lr = mean_squared_error(target_valid_2, predictions_valid_lr_2)**0.5
print('Средний запас сырья:', predictions_mean, 'rmse =', result_lr)

Средний запас сырья: 68.17320024946817 rmse = 1.0231340913429756


In [24]:
model_lr_3 = LinearRegression()
model_lr_3.fit(features_train_3, target_train_3) 
predictions_valid_lr_3 = model_lr_3.predict(features_valid_3) 
predictions_mean = predictions_valid_lr_3.mean()
result_lr = mean_squared_error(target_valid_3, predictions_valid_lr_3)**0.5
print('Средний запас сырья:', predictions_mean, 'rmse =', result_lr)

Средний запас сырья: 94.46742884611454 rmse = 40.03077959471606


**Вывод:** Самый высокий средний запас сырья в третьем регионе, но там же самая высокая ошибка, во втором сырья меньше, но предсказания более точные, первая модель предсказала средний запас сырья относительно других моделей, так же ошибку ниже чем у модели построенной по третьему региону.

## Подготовка к расчёту прибыли

Далее введем и пропишем в переменных параметры заданные задачей, так же посчитаем достаточный средний объем сырья для окупаемости добычи.

In [25]:
income = 450000 # доход с единицы сырья столбца income
budget = 10000000000 # бюджет заложенный на разработку в регионе
units = 200 # количество скважин для разработки
enough_volume = budget/income/units # достаточный средний объем сырья в скважине
enough_volume

111.11111111111111

Достаточный средний объем для окупаемости 111 тыс баррелей на 200 разработанных скважинах это минимальный порог. Далее посчитаем прибыль по предсказанным на валидационной выборке результатам:

In [26]:
def profit(predictions, target, units):     # функция рассчета прибыли
    predictions = pd.Series(predictions, index=target.index)
    predictions_best = predictions.sort_values(ascending=False)
    target_best = target[predictions_best.index][:units]
    return sum(target_best)*income - budget

res_1 = profit(predictions_valid_lr_1, target_valid_1, units)
res_2 = profit(predictions_valid_lr_2, target_valid_2, units)
res_3 = profit(predictions_valid_lr_3, target_valid_3, units)
print('Прибыль в первом регионе:', res_1)
print('Прибыль во втором регионе:', res_2)
print('Прибыль в третьем регионе:', res_3)

Прибыль в первом регионе: 3276350000
Прибыль во втором регионе: 2330000000
Прибыль в третьем регионе: 2665700000


**Вывод:** Самая высокая прибыль предсказанная на валидационной выборке в третьем регионе, после него идет третий регион, самый низкий результат у второго региона.

## Расчёт прибыли и рисков 

С помощью техники bootstrap определим среднюю прибыль, возможные риски не получить прибыль, доверительный интервал:

In [27]:
def bootstrap(predictions, target): # функция для bootstrap с дальнейшим расчетом показателей 
    state = np.random.RandomState(0)
    values = []
 
    for i in range(1000):
        target_subsamples = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        predictions_subsamples = predictions[target_subsamples.index]
        values.append(profit(predictions_subsamples, target_subsamples, units))
        
    values = pd.Series(values)
    mean_profit = values.mean()
    loss = np.mean(values < 0)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return 'Средняя прибыль:', mean_profit, 'Риск:', loss, 'Нижний предел доверительного интервала:', lower, 'Верхний предел доверительного интервала:', upper
    
bootstrap_1 = bootstrap(predictions_valid_lr_1, target_valid_1) 
bootstrap_2 = bootstrap(predictions_valid_lr_2, target_valid_2) 
bootstrap_3 = bootstrap(predictions_valid_lr_3, target_valid_3) 
print(bootstrap_1)
print(bootstrap_2)
print(bootstrap_3)

('Средняя прибыль:', 365167250.0, 'Риск:', 0.097, 'Нижний предел доверительного интервала:', -208461250.0, 'Верхний предел доверительного интервала:', 926629999.9999999)
('Средняя прибыль:', 439985150.0, 'Риск:', 0.021, 'Нижний предел доверительного интервала:', 26292500.000000007, 'Верхний предел доверительного интервала:', 843728750.0)
('Средняя прибыль:', 367620200.0, 'Риск:', 0.099, 'Нижний предел доверительного интервала:', -176511250.0, 'Верхний предел доверительного интервала:', 901789999.9999999)


**Вывод:** Из выше произведенного анализа можно представить следующие данные по каждому региону: 

Первый регион:
* Предсказанный средний запас сырья: 92.0933978221753, квадратичная ошибка: 37.58106136304993;
* Возможная прибыль в регионе: 3276350000;
* Средняя прибыль: 365167250.0;
* Риск: 0.097; 
* Нижний предел доверительного интервала: -208461250.0, Верхний предел доверительного интервала: 926629999.9999999.

Второй регион: 
* Предсказанный средний запас сырья: 68.17320024946817, квадратичная ошибка: 1.0231340913429756;
* Возможная прибыль в регионе: 2330000000;
* Средняя прибыль: 439985150.0;
* Риск: 0.021;
* Нижний предел доверительного интервала: 26292500.000000007, Верхний предел доверительного интервала: 843728750.0.

Третий регион: 
* Предсказанный средний запас сырья: 94.46742884611454, квадратичная ошибка: 40.03077959471606;
* Возможная прибыль в регионе: 2665700000;
* Средняя прибыль: 367620200.0;
* Риск: 0.099; 
* Нижний предел доверительного интервала: -176511250.0, Верхний предел доверительного интервала: 901789999.9999999.

Из выше приведенных данных наиболее вероятным регионом для разработки является второй, так как имеет наиболее низкий показатель рисков 2.1%, условиями задачи верхний порог задан на отметке 2.5%, поэтому этот регион удовлетворяет данному условию. Хотя этот регион и обладает низким предсказанным средним запасом сырья, но при этом модель имеет здесь наименьшую ошибку.
